In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
from pyhive import presto

plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


import warnings
warnings.filterwarnings('ignore')

## Connection
connection = presto.connect(
        host='presto-gateway.serving.data.production.internal',
        port=80,
        protocol='http',
        catalog='hive',
        username='manoj.ravirajan@rapido.bike',
    )

### Rain_Days 

In [2]:
validation_from = '20230619'
validation_to   = '20230625'

city_list = [
    'Pune','Mumbai'
]

tuple(city_list)

('Pune', 'Mumbai')

## True Data  Rain

In [4]:
swiggy_rain_query = f"""
    SELECT
    *
    FROM
        hive.pricing.marketplace_iprice_rain_signals irs
    JOIN 
        datasets.city_cluster_hex cch 
        ON cch.cluster = irs.cluster_name 
        AND resolution = 8 AND cch.cluster != ''
    WHERE
        yyyymmdd between '{validation_from}' and '{validation_to}'
        AND source in ('SWIGGY_RAIN_SIGNAL','L2A_RAIN_SIGNAL')
        AND city in {tuple(city_list)}
"""

#print(swiggy_rain_query)

swiggy_rain_data = pd.read_sql(swiggy_rain_query, connection)

swiggy_rain_data.head()

DatabaseError: Execution failed on sql: SELECT
    *
    FROM
        hive.pricing.marketplace_iprice_rain_signals irs
    JOIN 
        datasets.city_cluster_hex cch 
        ON cch.cluster = irs.cluster_name 
        AND resolution = 8 AND cch.cluster != ''
    WHERE
        yyyymmdd between '20230619' and '20230625'
        AND source in ('SWIGGY_RAIN_SIGNAL','L2A_RAIN_SIGNAL')
        AND city in ('Pune', 'Mumbai')

HTTPConnectionPool(host='presto-gateway.serving.data.production.internal', port=80): Max retries exceeded with url: /v1/statement (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fe53d1841c0>, 'Connection to presto-gateway.serving.data.production.internal timed out. (connect timeout=None)'))
unable to rollback

In [ ]:
swiggy_rain_data.city.unique()

In [5]:
swiggy_rain_data_v1 = swiggy_rain_data[swiggy_rain_data['source']=='SWIGGY_RAIN_SIGNAL']

In [6]:
swiggy_rain_data_v1.head()

,type,source,created_on,updated_at,timestamp,yyyymmdd,quarter_hour,city_id,cluster_id,cluster_name,rain_level,precipitation_probability,city_name,city,cluster,hex_id,cluster_original,resolution,executiondate
0,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 18:33:30,2023-06-25 18:33:30,1687698210768,20230625,1830,5bc5acb112477c2ece769599,NewTown Action Area 1B,NewTown Action Area 1B,RAIN,80,None,Kolkata,NewTown Action Area 1B,883cf2134dfffff,NewTown Action Area 1B,8,2023-06-26
1,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 18:33:30,2023-06-25 18:33:30,1687698210768,20230625,1830,5bc5acb112477c2ece769599,NewTown Action Area 1B,NewTown Action Area 1B,RAIN,80,None,Kolkata,NewTown Action Area 1B,883cf2c69bfffff,NewTown Action Area 1B,8,2023-06-26
2,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 18:33:30,2023-06-25 18:33:30,1687698210768,20230625,1830,5bc5acb112477c2ece769599,NewTown Action Area 1B,NewTown Action Area 1B,RAIN,80,None,Kolkata,NewTown Action Area 1B,883cf2c699fffff,NewTown Action Area 1B,8,2023-06-26
3,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 22:46:21,2023-06-25 22:46:21,1687713381241,20230625,2300,572ca7ff116b5db3057bd814,Yelahanka New Town,Yelahanka New Town,RAIN,80,None,Bangalore,Yelahanka New Town,8860169647fffff,Yelahanka New Town,8,2023-06-26
4,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 22:46:21,2023-06-25 22:46:21,1687713381241,20230625,2300,572ca7ff116b5db3057bd814,Yelahanka New Town,Yelahanka New Town,RAIN,80,None,Bangalore,Yelahanka New Town,8860169655fffff,Yelahanka New Town,8,2023-06-26


In [7]:
import h3
def get_h6_from_h8(hex_id):
    if h3.h3_is_valid(hex_id):
        h8_geo = h3.h3_to_geo(hex_id)
        h6     = h3.geo_to_h3(lat=h8_geo[0],
                              lng=h8_geo[1],
                              resolution = 6
                             )
    return h6

In [8]:
swiggy_rain_data_v1['hex_6'] = swiggy_rain_data_v1.hex_id.apply(lambda x: get_h6_from_h8(x))

swiggy_rain_data_v1.head()

,type,source,created_on,updated_at,timestamp,yyyymmdd,quarter_hour,city_id,cluster_id,cluster_name,rain_level,precipitation_probability,city_name,city,cluster,hex_id,cluster_original,resolution,executiondate,hex_6
0,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 18:33:30,2023-06-25 18:33:30,1687698210768,20230625,1830,5bc5acb112477c2ece769599,NewTown Action Area 1B,NewTown Action Area 1B,RAIN,80,None,Kolkata,NewTown Action Area 1B,883cf2134dfffff,NewTown Action Area 1B,8,2023-06-26,863cf2137ffffff
1,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 18:33:30,2023-06-25 18:33:30,1687698210768,20230625,1830,5bc5acb112477c2ece769599,NewTown Action Area 1B,NewTown Action Area 1B,RAIN,80,None,Kolkata,NewTown Action Area 1B,883cf2c69bfffff,NewTown Action Area 1B,8,2023-06-26,863cf2c6fffffff
2,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 18:33:30,2023-06-25 18:33:30,1687698210768,20230625,1830,5bc5acb112477c2ece769599,NewTown Action Area 1B,NewTown Action Area 1B,RAIN,80,None,Kolkata,NewTown Action Area 1B,883cf2c699fffff,NewTown Action Area 1B,8,2023-06-26,863cf2c6fffffff
3,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 22:46:21,2023-06-25 22:46:21,1687713381241,20230625,2300,572ca7ff116b5db3057bd814,Yelahanka New Town,Yelahanka New Town,RAIN,80,None,Bangalore,Yelahanka New Town,8860169647fffff,Yelahanka New Town,8,2023-06-26,866016967ffffff
4,rainSignal,SWIGGY_RAIN_SIGNAL,2023-06-25 22:46:21,2023-06-25 22:46:21,1687713381241,20230625,2300,572ca7ff116b5db3057bd814,Yelahanka New Town,Yelahanka New Town,RAIN,80,None,Bangalore,Yelahanka New Town,8860169655fffff,Yelahanka New Town,8,2023-06-26,866016967ffffff


In [9]:
swiggy_rain_data_v1.groupby(['city','city_id']).source.count()

city           city_id                 
Bangalore      572ca7ff116b5db3057bd814    73786
Chennai        57af2db19729ad145ddbba66    32280
Coimbatore     5a5f74f5b3f3978342561e99     3371
Delhi          5bc5ac2312477c2ece769591    39497
Guwahati       5bc5adea12477c2ece7695ac      257
Hyderabad      5740135d4fdf4798208bba24    68905
Jaipur         5ba090686fde19440c388a07     5008
Kolkata        5bc5acb112477c2ece769599     5401
Tirupati       5c134ebb7a6187300d0c0f97      249
Vishakapatnam  5a5f70a81635168040c9a163     1425
Name: source, dtype: int64

In [10]:
swiggy_rain_data_v1[swiggy_rain_data_v1['hex_6']=='863c9300fffffff'].hex_id.unique()

array(['883c9300e7fffff', '883c9300a9fffff', '883c9300e5fffff',
       '883c9300adfffff', '883c9300ebfffff', '883c930085fffff',
       '883c9300e3fffff', '883c9300e1fffff', '883c9300e9fffff',
       '883c93008dfffff', '883c9300c7fffff', '883c9300c5fffff',
       '883c930089fffff', '883c9300ddfffff', '883c930099fffff',
       '883c9300d1fffff', '883c9300cbfffff', '883c9300d5fffff',
       '883c9300d7fffff', '883c930083fffff', '883c930087fffff',
       '883c93008bfffff', '883c930095fffff', '883c93009dfffff',
       '883c930081fffff'], dtype=object)

In [11]:
swiggy_rain_data_v1.quarter_hour.min()

'0000'

In [12]:
city_rain_sumary = swiggy_rain_data_v1[swiggy_rain_data_v1['quarter_hour']>='0600'].groupby(['city','yyyymmdd','hex_6']).quarter_hour.nunique().reset_index()
city_rain_sumary['hour'] = city_rain_sumary['quarter_hour']/4
city_rain_sumary.head(2)

,city,yyyymmdd,hex_6,quarter_hour,hour
0,Bangalore,20230619,866014407ffffff,2,0.5
1,Bangalore,20230619,866014417ffffff,2,0.5


In [13]:
city_rain_day = swiggy_rain_data_v1.groupby(['city','yyyymmdd']).quarter_hour.nunique().reset_index()
city_rain_day['hour'] = city_rain_day['quarter_hour']/4
city_rain_day

,city,yyyymmdd,quarter_hour,hour
0,Bangalore,20230619,23,5.75
1,Bangalore,20230620,63,15.75
2,Bangalore,20230621,59,14.75
3,Bangalore,20230622,38,9.50
4,Bangalore,20230623,22,5.50
5,Bangalore,20230624,22,5.50
6,Bangalore,20230625,21,5.25
7,Chennai,20230619,86,21.50
8,Chennai,20230620,33,8.25
9,Chennai,20230621,29,7.25


In [14]:
city_rain_validations = swiggy_rain_data_v1.groupby(['city','yyyymmdd','quarter_hour','hex_6'],as_index=False).rain_level.count()
city_rain_validations['rain_flag'] = city_rain_validations['rain_level'].apply(lambda x : 1 if x>0 else 0)

city_rain_validations.rename(columns={'city':'city_name',
                                      'quarter_hour':'time_level',
                                      'hex_6':'geo_level'},inplace=True)

pd.concat([city_rain_validations.head(),city_rain_validations.tail()])

,city_name,yyyymmdd,time_level,geo_level,rain_level,rain_flag
0,Bangalore,20230619,1515,866014587ffffff,8,1
1,Bangalore,20230619,1515,866014597ffffff,4,1
2,Bangalore,20230619,1515,86601459fffffff,5,1
3,Bangalore,20230619,1530,866014587ffffff,28,1
4,Bangalore,20230619,1530,866014597ffffff,4,1
14595,Vishakapatnam,20230624,1330,863c9310fffffff,1,1
14596,Vishakapatnam,20230624,1330,863c9311fffffff,25,1
14597,Vishakapatnam,20230624,1330,863c93187ffffff,11,1
14598,Vishakapatnam,20230624,1330,863c9318fffffff,4,1
14599,Vishakapatnam,20230624,1330,863c931afffffff,40,1
